In [25]:
import yaml

def load_resume_data(yaml_file_path="resume.yaml"):
    """Loads and parses the resume.yaml file."""
    with open(yaml_file_path, "r") as f:
        try:
            resume_data = yaml.safe_load(f)
            return resume_data
        except yaml.YAMLError as e:
            print(f"Error parsing YAML file: {e}")
            return None

def extract_heading_section(resume_data):
    """Extracts the heading section from resume data."""
    personal_info = resume_data.get("personal_information", {})
    heading_data = {
        "name": f"{personal_info.get('name', '')} {personal_info.get('surname', '')}",
        "phone": f"{personal_info.get('phone_prefix', '')}{personal_info.get('phone', '')}",
        "email": personal_info.get("email", ""),
        "linkedin": personal_info.get("linkedin", ""),
        "github": personal_info.get("github", ""),
    }
    return heading_data

def extract_education_section(resume_data):
    """Extracts the education section from resume data."""
    education_list = resume_data.get("education_details", [])
    education_data = []
    for edu in education_list:
        education_data.append({
            "university": edu.get("university", ""),
            "duration": edu.get("graduation_year", ""),
            "degree": f"{edu.get('degree', '')}s in {edu.get('field_of_study', '')}",
            "location": edu.get("location", ""),  # You don't have location in your education details
            "gpa": edu.get("gpa", ""),
            "relevant_coursework": edu.get("courses", [])
        })
    return education_data

def extract_experience_section(resume_data):
    """Extracts the experience section from resume data."""
    experience_list = resume_data.get("experience_details", [])
    experience_data = []
    for exp in experience_list:
        experience_data.append({
            "company": exp.get("company", ""),
            "duration": exp.get("employment_period", ""),
            "role": exp.get("position", ""),
            "location": exp.get("location", ""),
            "tools": exp.get("skills_acquired", []),
            "responsibilities": exp.get("key_responsibilities", [])
        })
    return experience_data

def extract_projects_section(resume_data):
    """Extracts the projects section from resume data."""
    project_list = resume_data.get("projects", [])
    project_data = []
    for proj in project_list:
        project_data.append({
            "title": proj.get("name", ""),
            "skills": [],  # You don't have specific skills listed per project, you could extract from description
            "descriptions": [proj.get("description", "")] # Assuming description is a single string, adjust if it's a list
        })
    return project_data

def extract_skills_section(resume_data):
    """Extracts the skills section from resume data."""
    skills_data = resume_data.get("skills", {})
    formatted_skills = []
    for category, skills in skills_data.items():
        formatted_skills.append({
            "category": category.replace("_", " ").title(),
            "items": skills
        })
    return formatted_skills



In [26]:
resume_file_path = "assets/resume.yaml"
resume_data = load_resume_data(resume_file_path)

education_data = extract_education_section(resume_data=resume_data)
experience_data = extract_experience_section(resume_data=resume_data)
projects_data = extract_projects_section(resume_data=resume_data)
skills_data = extract_skills_section(resume_data=resume_data)
heading_data = extract_heading_section(resume_data=resume_data)

In [27]:
heading_data

{'name': 'Bhavya Pranav Tandra',
 'phone': '+16179062441',
 'email': 'tandra.b@northeastern.edu',
 'linkedin': 'https://www.linkedin.com/in/pranav-tandra525',
 'github': 'https://github.com/Pranavbp525'}

In [28]:
from pydantic import BaseModel
class HeadingData(BaseModel):
    name: str
    phone: str
    email: str
    linkedin: str
    github: str

In [29]:
heading = HeadingData.model_validate(heading_data)

In [30]:
heading

HeadingData(name='Bhavya Pranav Tandra', phone='+16179062441', email='tandra.b@northeastern.edu', linkedin='https://www.linkedin.com/in/pranav-tandra525', github='https://github.com/Pranavbp525')

In [31]:
education_data

[{'university': 'Northeastern University',
  'duration': '2025',
  'degree': 'Masterss in Artificial Intelligence',
  'location': 'Boston, MA',
  'gpa': '3.76/4.00',
  'relevant_coursework': ['Large Language Models',
   'Deep Learning',
   'Machine Learning',
   'Foundations in Artificial Intelligence',
   'Artificial Intelligence for Human Computer Interaction',
   'Algorithms',
   'Programming Design Paradigm']},
 {'university': 'Malla Reddy College of Engineering and Technology',
  'duration': '2022',
  'degree': 'Bachelorss in Computer Science',
  'location': 'Hyderabad, India',
  'gpa': '8.43/10.00',
  'relevant_coursework': ['Data Warehousing and Data Mining',
   'Cloud Computing',
   'Linux',
   'Big Data Analytics',
   'Data Visualization',
   'Parallel and Distributed Computing',
   'Database Management Systems']}]

In [32]:
def education_prompt(job_description, user_education):
    """Generates a prompt for the education section."""
    
    schema_instruction = """
    {
        "university": "Name of the university",
        "duration": "Years attended",
        "degree": "Degree earned",
        "location": "Location of the university",
        "gpa": "Grade point average (optional)",
        "relevant_coursework": ["List of relevant courses (optional)"]
    }
    """
    
    prompt = f"""
    Job Description:
    {job_description}

    My education:
    {user_education}

    Please select up to 8 relevant courses from my education history that match the job description.
    Output my education details in JSON format adhering to the following schema:

    {schema_instruction}

    Ensure the output is a valid JSON array with each element representing an education entry adhering to the specified schema.
    """
    return prompt

In [45]:
education_prompt = education_prompt("", education_response)

TypeError: 'str' object is not callable

In [44]:
education_response = EducationResponse(education=[EducationItem(**item) for item in education_data])


In [36]:
from typing import List, Optional
class EducationItem(BaseModel):
    university: str
    duration: str
    degree: str
    location: str
    gpa: Optional[str] = None  # GPA is optional
    relevant_coursework: Optional[List[str]] = None  # Relevant coursework is optional

class EducationResponse(BaseModel):
    education: List[EducationItem]

In [41]:
x = [EducationItem.model_validate(education) for education in education_data]

In [42]:
x

[EducationItem(university='Northeastern University', duration='2025', degree='Masterss in Artificial Intelligence', location='Boston, MA', gpa='3.76/4.00', relevant_coursework=['Large Language Models', 'Deep Learning', 'Machine Learning', 'Foundations in Artificial Intelligence', 'Artificial Intelligence for Human Computer Interaction', 'Algorithms', 'Programming Design Paradigm']),
 EducationItem(university='Malla Reddy College of Engineering and Technology', duration='2022', degree='Bachelorss in Computer Science', location='Hyderabad, India', gpa='8.43/10.00', relevant_coursework=['Data Warehousing and Data Mining', 'Cloud Computing', 'Linux', 'Big Data Analytics', 'Data Visualization', 'Parallel and Distributed Computing', 'Database Management Systems'])]

In [43]:
EducationResponse.model_validate(x)

ValidationError: 1 validation error for EducationResponse
  Input should be a valid dictionary or instance of EducationResponse [type=model_type, input_value=[EducationItem(university...e Management Systems'])], input_type=list]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type

In [46]:
education_response

EducationResponse(education=[EducationItem(university='Northeastern University', duration='2025', degree='Masterss in Artificial Intelligence', location='Boston, MA', gpa='3.76/4.00', relevant_coursework=['Large Language Models', 'Deep Learning', 'Machine Learning', 'Foundations in Artificial Intelligence', 'Artificial Intelligence for Human Computer Interaction', 'Algorithms', 'Programming Design Paradigm']), EducationItem(university='Malla Reddy College of Engineering and Technology', duration='2022', degree='Bachelorss in Computer Science', location='Hyderabad, India', gpa='8.43/10.00', relevant_coursework=['Data Warehousing and Data Mining', 'Cloud Computing', 'Linux', 'Big Data Analytics', 'Data Visualization', 'Parallel and Distributed Computing', 'Database Management Systems'])])